In [1]:
from ortools.constraint_solver import pywrapcp

In [2]:
# create the solver
solver = pywrapcp.Solver('jobshop')

machines_count = 3
jobs_count = 3
all_machines = range(0, machines_count)
all_jobs = range(0, jobs_count)
# define data.
machines = [[0, 1, 2],
            [0, 2, 1],
            [1, 2]]

processing_times = [[3, 2, 2],
                   [2, 1, 4],
                   [4, 3]]


In [3]:
solver = pywrapcp.Solver('jobshop')

In [4]:
# computes horizon.
horizon = 0
for i in all_jobs:
    horizon += sum(processing_times[i])

In [6]:
# create jobs
all_tasks = {}
for i in all_jobs:
    for j in range(0, len(machines[i])):
        all_tasks[(i, j)] = solver.FixedDurationIntervalVar(
            0,
            horizon,
            processing_times[i][j],
            False,
            'Job_%i_%i' % (i, j)
        )
        
# creates sequence variables and add disjunctive constraints.
all_sequences = []
all_machines_jobs = []
for i in all_machines:
    machines_jobs = []
    for j in all_jobs:
        for k in range(0, len(machines[j])):
            if machines[j][k] == i:
                machines_jobs.append(all_tasks[(j, k)])
    disj = solver.DisjunctiveConstraint(machines_jobs, 'machine %i' % i)
    all_sequences.append(disj.SequenceVar())
    solver.Add(disj)
    
# Add conjunctive contraints.
for i in all_jobs:
    for j in range(0, len(machines[i]) - 1):
        solver.Add(all_tasks[(i, j + 1)].StartsAfterEnd(all_tasks[(i, j)]))


In [7]:
# define the objective
# set the objective
obj_var = solver.Max([all_tasks[(i, len(machines[i])-1)].EndExpr() for i in all_jobs])

In [8]:
# Create search phases.
sequence_phase = solver.Phase([all_sequences[i] for i in all_machines],
                                solver.SEQUENCE_DEFAULT)
vars_phase = solver.Phase([obj_var],
                            solver.CHOOSE_FIRST_UNBOUND,
                            solver.ASSIGN_MIN_VALUE)
main_phase = solver.Compose([sequence_phase, vars_phase])